In [2]:
library(dplyr)
library(tidyverse)
library(reshape2)
library(gridExtra)
library(grid)

In [3]:
code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-1)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "pl_functions.R"))
base_dir <- gsub("/plotting_codes", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC"

In [4]:
exp_name <- "0_Acute-Chronic"
analysis_name <- c("0", "resampled") 

use.path <- file.path(base_dir, '9_Figures', exp_name, 
                      paste(analysis_name, collapse="_"),
                      'node_plots')
dir.create(use.path, showWarnings = FALSE, recursive=TRUE)
setwd(use.path)

## Proprocess

In [5]:
#####---------- Pre-process paga connection file
if (FALSE) {
  paga.out.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/0_Acute-Chronic_paga'
  paga.conn.file <- "paga_connect.csv"
  paga.conn.file <- file.path(paga.out.dir, paga.conn.file)
  paga.conn.file.long <- gsub(".csv", "_long.csv", basename(paga.conn.file))
    
  new_names <- file.path(base_dir,'/0_Acute-Chronic/2_scVelo/0_scVelo_out_resampled/pt_ordered_cluster_avg.csv')
  new_names_df <- read_csv(new_names) %>% dplyr::select(one_of(c('louvain', 'louvain_new_label')))
    
  #--- Read input
  paga.conn.tb <- read_csv(paga.conn.file)
  paga.conn.tb <- paga.conn.tb %>% melt(id.vars="X1") # Melt wide to long
  colnames(paga.conn.tb) <- c("From", "To", "Conn")
  #--- Remove duplicates (x_y & y_x keep one only)
  paga.conn.tb$From <- as.numeric(paga.conn.tb$From)
  paga.conn.tb$To <- as.numeric(as.character(paga.conn.tb$To))
  paga.conn.tb <- paga.conn.tb %>% rowwise() %>% mutate(fromto = paste(as.character(sort(c(From, To))), collapse="_"))
  paga.conn.tb$From <- NULL
  paga.conn.tb$To <- NULL
  paga.conn.tb <- paga.conn.tb %>% distinct()
  #--- Separate x_y into x y
  paga.conn.tb <- paga.conn.tb %>% separate(fromto, c("node1", "node2"), sep="_")
  paga.conn.tb$node1 <- as.numeric(paga.conn.tb$node1)
  paga.conn.tb$node2 <- as.numeric(paga.conn.tb$node2)
  paga.conn.tb <- paga.conn.tb %>% filter(Conn > 0)
  #--- Convert new names
  node1_df <- new_names_df
  colnames(node1_df) <- c("node1", "node1_new") 
  node2_df <- new_names_df
  colnames(node2_df) <- c("node2", "node2_new") 
  paga.conn.tb <- paga.conn.tb %>% 
    left_join(node1_df) %>% left_join(node2_df) 
  colnames(paga.conn.tb) <- c("Conn", "node1_old", "node2_old", "node1", "node2")
  
  #--- Save
  write_csv(paga.conn.tb, paga.conn.file.long)
}

# Get coordinates for edges
if (FALSE) {
  paga.out.dir <- '/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic/1_Scanpy/0_Scanpy_out_resampled/0_Acute-Chronic_paga'
  paga.pos.file <- "paga_pos.csv"
  paga.pos.file <- file.path(paga.out.dir, paga.pos.file)
  paga.conn.file.coor <- gsub(".csv", "_coor.csv", paga.conn.file.long)

  new_names <- file.path(base_dir,'/0_Acute-Chronic/2_scVelo/0_scVelo_out_resampled/pt_ordered_cluster_avg.csv')
  new_names_df <- read_csv(new_names) %>% dplyr::select(one_of(c('louvain', 'louvain_new_label')))
    
  paga.conn.file.long <- "paga_connect_long.csv"
  paga.pos.tb <- read_csv(paga.pos.file)
  paga.conn.tb <- read_csv(paga.conn.file.long)
  
  #--- Merge dataframe with connection & coordinates
  colnames(paga.pos.tb) <- c("node1_old", "x1", "y1")
  paga.conn.tb <- paga.conn.tb %>% left_join(paga.pos.tb)
  colnames(paga.pos.tb) <- c("node2_old", "x2", "y2")
  paga.conn.tb <- paga.conn.tb %>% left_join(paga.pos.tb)
  write_csv(paga.conn.tb, paga.conn.file.coor)
    
  #--- Convert node position dataframe to new name
  paga.pos.file.new <- "paga_pos_new.csv"
  colnames(paga.pos.tb) <- c('louvain', 'x', 'y')
  paga.pos.tb <- paga.pos.tb %>% left_join(new_names_df)
  colnames(paga.pos.tb) <- c("louvain_old", "x", "y", "louvain")
  write_csv(paga.pos.tb, paga.pos.file.new)
}

# 1. Plot 

In [6]:
paga.conn.tb <- read_csv('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/0_Acute-Chronic/0_resampled/node_plots/paga_connect_long_coor.csv')
paga.pos.tb <- read_csv('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/0_Acute-Chronic/0_resampled/node_plots/paga_pos_new.csv')
paga.conn.tb <- paga.conn.tb %>% filter(Conn > 0.25)

Parsed with column specification:
cols(
  Conn = col_double(),
  node1_old = col_double(),
  node2_old = col_double(),
  node1 = col_character(),
  node2 = col_character(),
  x1 = col_double(),
  y1 = col_double(),
  x2 = col_double(),
  y2 = col_double()
)

Parsed with column specification:
cols(
  louvain_old = col_double(),
  x = col_double(),
  y = col_double(),
  louvain = col_character()
)



## Seperate ARM / Cl13 GSEA output

In [31]:
wk_dir <- file.path(base_dir, '/9_Figures/0_Acute-Chronic/0_resampled/louvain_celltype_general_withNAV_sum_plot')
setwd(wk_dir)
source(file.path(tool_dir, "pl_functions.R"))

In [32]:
gsea.out.file <- 'louvain_celltype_general_withNAV_gsea_all.csv'
gsea.out.df <- read_csv(gsea.out.file)

gs.use.df <- read_csv('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/Y_annotated/anno_plotuse_gs_20200928.csv')
gs.use.df <- gs.use.df %>% drop_na(plot_use)

Parsed with column specification:
cols(
  comparison = col_character(),
  pathway = col_character(),
  NES = col_double(),
  padj = col_double(),
  leadingEdge_signal = col_double(),
  mlog10padj = col_double(),
  old_names = col_character()
)

Parsed with column specification:
cols(
  gs_name = col_character(),
  type = col_character(),
  use = col_character(),
  plot_use = col_character(),
  abbr = col_character(),
  order = col_double()
)



In [33]:
for (i in c(1: nrow(gs.use.df))) {
    gs_i <- gs.use.df$gs_name[i]
    gs_abbr_i <- gs.use.df$abbr[i]

    plot.i.df <- gsea.out.df %>% filter(pathway == gs_i) %>% separate(comparison, c("louvain", "celltype"))
    plot.i.df$nlog10padj <- -log10(plot.i.df$padj)

    plot.i.a.df <- plot.i.df %>% filter(celltype == 'A') %>% select(one_of(c("louvain", "NES", "nlog10padj")))
    colnames(plot.i.a.df) <- c("louvain", "val", "size")
    plot.i.a.df <- paga.pos.tb %>% left_join(plot.i.a.df)

    plot.i.c.df <- plot.i.df %>% filter(celltype == 'C') %>% select(one_of(c("louvain", "NES", "nlog10padj")))
    colnames(plot.i.c.df) <- c("louvain", "val", "size")
    plot.i.c.df <- paga.pos.tb %>% left_join(plot.i.c.df)

    conn_plot(plot.i.a.df, paga.conn.tb, paste(gs_abbr_i, "(Arm)"), "NES" , "white", mute=FALSE)
    conn_plot(plot.i.c.df, paga.conn.tb, paste(gs_abbr_i, "(Cl13)"), "NES" , "white", mute=FALSE)    
}

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

Joining, by = "louvain"

